In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

import matplotlib.pyplot as plt
import numpy as np 

from compare_f1_f2.results_compare import Facts1Results, Facts2Results, check_ice_sheet_projections
from compare_f1_f2.plotting import plot_f1_f2_diffs_for_icesheet

In [6]:
import xarray as xr
import os

def read_netcdf_files(data_dir):
    """
    Reads all NetCDF files in the specified directory and returns a dictionary of xarray Datasets.
    The keys are filenames, and the values are the loaded Datasets.
    """
    datasets = {}
    for fname in os.listdir(data_dir):
        if fname.endswith('.nc'):
            fpath = os.path.join(data_dir, fname)
            ds = xr.open_dataset(fpath)
            datasets[fname] = ds
    return datasets


In [12]:
scenarios_ls = ["rcp26","rcp45","rcp85",
             "ssp126","ssp245","ssp585"]

icesheets_ls = ["ais","eais","wais"]
levels_ls = ["global","local"]

icesheets_f1_map = {
    "ais":"AIS",
    "eais":"EAIS",
    "wais":"WAIS"
}
icesheets_f2_map = {
    "ais":"ais",
    "eais":"eais",
    "wais":"wais"
}

levels_f1_map = {
    "global":"global",
    "local":"local" 
}
levels_f2_map = {
    "global":"gs",
    "local":"ls"
}

# Create nested dict: mydict[scenario][level][icesheet] = {'f1': f1_path, 'f2': f2_path}
mydict = {}
for scenario in scenarios_ls:
    mydict[scenario] = {}
    for level in levels_ls:
        mydict[scenario][level] = {}
        for icesheet in icesheets_ls:
            f1_icesheet = icesheets_f1_map[icesheet]
            f1_level = levels_f1_map[level]
            f1_gen_path = f"~/Desktop/facts_work/facts_v1/facts/experiments/deconto21.{scenario}/output/deconto21.{scenario}.deconto21.deconto21.ais_{f1_icesheet}_{f1_level}sl.nc"

            f2_icesheet = icesheets_f2_map[icesheet]
            f2_level = levels_f2_map[level]
            f2_dir_gen_path = f"~/Desktop/facts_work/facts_v2/deconto21-ais/data/output/{scenario}_no_climate/{f2_icesheet}_{f2_level}lr.nc"
            
            f1_ds = xr.open_dataset(f1_gen_path)
            f2_ds = xr.open_dataset(f2_dir_gen_path)

            assert f1_ds['sea_level_change'].equals(
                f2_ds['sea_level_change']
            ) == True, f"Sea level rise mismatch for {scenario} {icesheet} {level}"

            # Store paths in dict
            mydict[scenario][level][icesheet] = {'f1': f1_gen_path, 'f2': f2_dir_gen_path}


In [14]:
ssp126_global_ais_f1_ds = xr.open_dataset(mydict['ssp126']['global']['ais']['f1'])
ssp126_global_ais_f2_ds = xr.open_dataset(mydict['ssp126']['global']['ais']['f2'])

In [17]:
ssp126_global_ais_f2_ds['sea_level_change'].mean()

<xarray.DataArray 'sea_level_change' ()> Size: 4B
array(91.34588, dtype=float32)

In [18]:
ssp126_global_ais_f1_ds['sea_level_change'].mean()

<xarray.DataArray 'sea_level_change' ()> Size: 4B
array(91.34588, dtype=float32)

In [84]:
# Example usage:
f2_rcp26_output_dir = os.path.expanduser("~/Desktop/facts_work/facts_v2/deconto21-ais/data/output/rcp26_no_climate_validation")
f2_rcp26_output_datasets = read_netcdf_files(f2_rcp26_output_dir)

f1_rcp26_output_dir = os.path.expanduser("~/Desktop/facts_work/facts_v1/facts/experiments/deconto21.rcp26/output")
f1_rcp26_output_datasets = read_netcdf_files(f1_rcp26_output_dir)

# Example usage:
f2_rcp45_output_dir = os.path.expanduser("~/Desktop/facts_work/facts_v2/deconto21-ais/data/output/rcp45_no_climate_validation")
f2_rcp45_output_datasets = read_netcdf_files(f2_rcp45_output_dir)

f1_rcp45_output_dir = os.path.expanduser("~/Desktop/facts_work/facts_v1/facts/experiments/deconto21.rcp45/output")
f1_rcp45_output_datasets = read_netcdf_files(f1_rcp45_output_dir)

# rcp8.5
f2_rcp85_output_dir = os.path.expanduser("~/Desktop/facts_work/facts_v2/deconto21-ais/data/output/rcp85_no_climate_validation")
f2_rcp85_output_datasets = read_netcdf_files(f2_rcp85_output_dir)

f1_rcp85_output_dir = os.path.expanduser("~/Desktop/facts_work/facts_v1/facts/experiments/deconto21.rcp85/output")
f1_rcp85_output_datasets = read_netcdf_files(f1_rcp85_output_dir)


In [70]:
print(list(f1_rcp26_output_datasets.keys()))

['deconto21.rcp26.deconto21.deconto21.ais_EAIS_globalsl.nc', 'deconto21.rcp26.deconto21.deconto21.ais_AIS_globalsl.nc', 'deconto21.rcp26.deconto21.deconto21.ais_AIS_localsl.nc', 'deconto21.rcp26.deconto21.deconto21.ais_EAIS_localsl.nc', 'deconto21.rcp26.deconto21.deconto21.ais_WAIS_globalsl.nc', 'deconto21.rcp26.deconto21.deconto21.ais_WAIS_localsl.nc']


In [71]:
print(list(f2_rcp26_output_datasets.keys()))

['ais_lslr.nc', 'eais_gslr.nc', 'wais_lslr.nc', 'ais_gslr.nc', 'eais_lslr.nc', 'wais_gslr.nc']


## RCP 2.6 -- no climate

In [72]:
f1_rcp26_output_datasets['deconto21.rcp26.deconto21.deconto21.ais_WAIS_globalsl.nc']['sea_level_change'].equals(
    f2_rcp26_output_datasets['wais_gslr.nc']['sea_level_change']
)

True

In [73]:
f1_rcp26_output_datasets['deconto21.rcp26.deconto21.deconto21.ais_EAIS_globalsl.nc']['sea_level_change'].equals(
    f2_rcp26_output_datasets['eais_gslr.nc']['sea_level_change']
)

True

In [74]:
f1_rcp26_output_datasets['deconto21.rcp26.deconto21.deconto21.ais_AIS_globalsl.nc']['sea_level_change'].equals(
    f2_rcp26_output_datasets['ais_gslr.nc']['sea_level_change']
)

True

In [75]:
f1_rcp26_output_datasets['deconto21.rcp26.deconto21.deconto21.ais_WAIS_localsl.nc']['sea_level_change'].equals(
    f2_rcp26_output_datasets['wais_lslr.nc']['sea_level_change']
)

True

In [76]:
f1_rcp26_output_datasets['deconto21.rcp26.deconto21.deconto21.ais_EAIS_localsl.nc']['sea_level_change'].equals(
    f2_rcp26_output_datasets['eais_lslr.nc']['sea_level_change']
)

True

In [77]:
f1_rcp26_output_datasets['deconto21.rcp26.deconto21.deconto21.ais_AIS_localsl.nc']['sea_level_change'].equals(
    f2_rcp26_output_datasets['ais_lslr.nc']['sea_level_change']
)

True

In [78]:
def compare_f1_f2_datasets(f1_datasets, f2_datasets, mapping, var_name='sea_level_change'):
    """
    Compares the specified variable in each mapped dataset from f1 and f2 using .equals().
    Returns a dictionary with keys as dataset names and values as the result of the comparison (True/False).
    mapping: dict where keys are f1 dataset names, values are f2 dataset names
    var_name: variable to compare (default 'sea_level_change')
    """
    results = {}
    for f1_name, f2_name in mapping.items():
        f1_var = f1_datasets[f1_name][var_name]
        f2_var = f2_datasets[f2_name][var_name]
        results[f1_name] = f1_var.equals(f2_var)
    return results


In [80]:
# Example mapping for RCP2.6 scenario
rcp26_mapping = {
    'deconto21.rcp26.deconto21.deconto21.ais_WAIS_globalsl.nc': 'wais_gslr.nc',
    'deconto21.rcp26.deconto21.deconto21.ais_EAIS_globalsl.nc': 'eais_gslr.nc',
    'deconto21.rcp26.deconto21.deconto21.ais_AIS_globalsl.nc': 'ais_gslr.nc',
    'deconto21.rcp26.deconto21.deconto21.ais_WAIS_localsl.nc': 'wais_lslr.nc',
    'deconto21.rcp26.deconto21.deconto21.ais_EAIS_localsl.nc': 'eais_lslr.nc',
    'deconto21.rcp26.deconto21.deconto21.ais_AIS_localsl.nc': 'ais_lslr.nc',
}

In [81]:
rcp26_comparison_results = compare_f1_f2_datasets(f1_rcp26_output_datasets, f2_rcp26_output_datasets, rcp26_mapping)
print(rcp26_comparison_results)

{'deconto21.rcp26.deconto21.deconto21.ais_WAIS_globalsl.nc': True, 'deconto21.rcp26.deconto21.deconto21.ais_EAIS_globalsl.nc': True, 'deconto21.rcp26.deconto21.deconto21.ais_AIS_globalsl.nc': True, 'deconto21.rcp26.deconto21.deconto21.ais_WAIS_localsl.nc': True, 'deconto21.rcp26.deconto21.deconto21.ais_EAIS_localsl.nc': True, 'deconto21.rcp26.deconto21.deconto21.ais_AIS_localsl.nc': True}


In [82]:
# Mapping dictionaries for other scenarios
rcp45_mapping = {
    'deconto21.rcp45.deconto21.deconto21.ais_WAIS_globalsl.nc': 'wais_gslr.nc',
    'deconto21.rcp45.deconto21.deconto21.ais_EAIS_globalsl.nc': 'eais_gslr.nc',
    'deconto21.rcp45.deconto21.deconto21.ais_AIS_globalsl.nc': 'ais_gslr.nc',
    'deconto21.rcp45.deconto21.deconto21.ais_WAIS_localsl.nc': 'wais_lslr.nc',
    'deconto21.rcp45.deconto21.deconto21.ais_EAIS_localsl.nc': 'eais_lslr.nc',
    'deconto21.rcp45.deconto21.deconto21.ais_AIS_localsl.nc': 'ais_lslr.nc',
}

rcp85_mapping = {
    'deconto21.rcp85.deconto21.deconto21.ais_WAIS_globalsl.nc': 'wais_gslr.nc',
    'deconto21.rcp85.deconto21.deconto21.ais_EAIS_globalsl.nc': 'eais_gslr.nc',
    'deconto21.rcp85.deconto21.deconto21.ais_AIS_globalsl.nc': 'ais_gslr.nc',
    'deconto21.rcp85.deconto21.deconto21.ais_WAIS_localsl.nc': 'wais_lslr.nc',
    'deconto21.rcp85.deconto21.deconto21.ais_EAIS_localsl.nc': 'eais_lslr.nc',
    'deconto21.rcp85.deconto21.deconto21.ais_AIS_localsl.nc': 'ais_lslr.nc',
}

ssp585_mapping = {
    'deconto21.ssp585.deconto21.deconto21.ais_WAIS_globalsl.nc': 'wais_gslr.nc',
    'deconto21.ssp585.deconto21.deconto21.ais_EAIS_globalsl.nc': 'eais_gslr.nc',
    'deconto21.ssp585.deconto21.deconto21.ais_AIS_globalsl.nc': 'ais_gslr.nc',
    'deconto21.ssp585.deconto21.deconto21.ais_WAIS_localsl.nc': 'wais_lslr.nc',
    'deconto21.ssp585.deconto21.deconto21.ais_EAIS_localsl.nc': 'eais_lslr.nc',
    'deconto21.ssp585.deconto21.deconto21.ais_AIS_localsl.nc': 'ais_lslr.nc',
}


In [83]:
rcp45_comparison_results = compare_f1_f2_datasets(f1_rcp45_output_datasets, f2_rcp45_output_datasets, rcp45_mapping)
print(rcp45_comparison_results)

{'deconto21.rcp45.deconto21.deconto21.ais_WAIS_globalsl.nc': True, 'deconto21.rcp45.deconto21.deconto21.ais_EAIS_globalsl.nc': True, 'deconto21.rcp45.deconto21.deconto21.ais_AIS_globalsl.nc': True, 'deconto21.rcp45.deconto21.deconto21.ais_WAIS_localsl.nc': True, 'deconto21.rcp45.deconto21.deconto21.ais_EAIS_localsl.nc': True, 'deconto21.rcp45.deconto21.deconto21.ais_AIS_localsl.nc': True}


In [85]:
rcp85_comparison_results = compare_f1_f2_datasets(f1_rcp85_output_datasets, f2_rcp85_output_datasets, rcp85_mapping)
print(rcp85_comparison_results)

{'deconto21.rcp85.deconto21.deconto21.ais_WAIS_globalsl.nc': True, 'deconto21.rcp85.deconto21.deconto21.ais_EAIS_globalsl.nc': True, 'deconto21.rcp85.deconto21.deconto21.ais_AIS_globalsl.nc': True, 'deconto21.rcp85.deconto21.deconto21.ais_WAIS_localsl.nc': True, 'deconto21.rcp85.deconto21.deconto21.ais_EAIS_localsl.nc': True, 'deconto21.rcp85.deconto21.deconto21.ais_AIS_localsl.nc': True}


In [ ]:
def verify_f1_f2_outputs(f2_output_dir_stem, f1_output_dir_stem, scenarios, mappings, var_name='sea_level_change'):
    """
    For each scenario, loads f1 and f2 outputs and compares them using the provided mapping dicts.
    Returns a dict of results for each scenario.
    f2_output_dir_stem: base path to f2 outputs (e.g. '~/Desktop/facts_work/facts_v2/deconto21-ais/data/output')
    f1_output_dir_stem: base path to f1 outputs (e.g. '~/Desktop/facts_work/facts_v1/facts/experiments')
    scenarios: list of scenario names (e.g. ['rcp26', 'rcp45', 'rcp85', 'ssp585'])
    mappings: dict of mapping dicts for each scenario
    var_name: variable to compare
    """
    results = {}
    for scenario in scenarios:
        # Build output directories
        f2_dir = os.path.expanduser(f"{f2_output_dir_stem}/{scenario}_no_climate_validation")
        f1_dir = os.path.expanduser(f"{f1_output_dir_stem}/deconto21.{scenario}/output")
        # Load datasets
        f2_datasets = read_netcdf_files(f2_dir)
        f1_datasets = read_netcdf_files(f1_dir)
        # Get mapping
        mapping = mappings[scenario]
        # Compare
        scenario_results = compare_f1_f2_datasets(f1_datasets, f2_datasets, mapping, var_name=var_name)
        results[scenario] = scenario_results
    return results

# Example usage:
scenario_list = ['rcp26', 'rcp45', 'rcp85', 'ssp585']
mapping_dicts = {
    'rcp26': rcp26_mapping,
    'rcp45': rcp45_mapping,
    'rcp85': rcp85_mapping,
    'ssp585': ssp585_mapping,
}
f2_output_dir_stem = '~/Desktop/facts_work/facts_v2/deconto21-ais/data/output'
f1_output_dir_stem = '~/Desktop/facts_work/facts_v1/facts/experiments'
all_comparison_results = verify_f1_f2_outputs(f2_output_dir_stem, f1_output_dir_stem, scenario_list, mapping_dicts)
print(all_comparison_results)
